In [10]:
import pandas as pd
from pymystem3 import Mystem
import numpy as np
from gensim.models.phrases import Phraser, Phrases
from gensim.models import TfidfModel, Word2Vec, FastText
from gensim import corpora
import nltk
import matplotlib.pyplot as plt
from collections import namedtuple, Counter
import re
import glob
import os
import matplotlib.pyplot as plt

from sklearn.externals import joblib

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.graph_objs import *

import artm

%matplotlib inline

print(artm.version())

0.9.0


In [2]:
df = pd.read_csv('labirint.csv', error_bad_lines=False)

In [4]:
%%time
#df = pd.read_csv('test.csv')
df.fillna('', inplace=True)
df['comment'] = df.title + ' ' + df.content


drop_comment = []
alph = 'йцукенгшщзхъфывапролджэячсмитьбю'
for i in range(len(df)):
    delete = False
    for char in df.comment.loc[i]:
        if char in alph:
            delete = False
            break
        else:
            delete = True
    if delete:
        drop_comment.append(i)
df.drop(drop_comment,axis=0,inplace=True)
df.index = range(len(df))


df['lemma'] = ''

m = Mystem()

idx = 0
for comment in df.comment.as_matrix():
    lemma = m.lemmatize(comment)[:]
    df.lemma.loc[idx] = ''.join(lemma[1:len(lemma)- 1])
    if idx%20000 == 0:
        print(idx)
    idx +=1

0
CPU times: user 833 ms, sys: 48.9 ms, total: 882 ms
Wall time: 3.45 s


In [5]:
stop_word = ["в",'!','?' "без", "до", "из", "к", "на", "по", "о", "от", "перед",
             "при", "через", "за", "над", "об", "под", "про", "для", "вблизи", 
             "вглубь", "вдоль", "возле", "около", "вокруг", "впереди", "после", 
             "посредством", "в","роли", "путём", "насчёт", "по","поводу", "ввиду", "по", "случаю", 
             "в", "течение", "благодаря", "несмотря", "на", "спустя", "с", "из-под", 
             "из-за", "я", "быть", "с", "это", "все", "то", "по-над", "отличие", "от", 
             "в","связи", "как", "словно", "так","как","т.к.", "тоже", "зато", "чтобы", 
             "также", "потому","что", "и", "а", "что", "или", "но", "однако", "когда", "лишь", 
             "едва", "где", "куда", "откуда", "столько", "настолько", "так", "такой","степени", "до",
             "того", "такой",  "будто", "точно", "если",  "коли", "ежели", "несмотря", "хотя", "хоть", 
             "пускай", "дабы", "-","же",":" "ли", "какой", "этот", "весь", "бы", "то", "у", 
             "уже","который","там","оно","ничто","кто","вот","тот","ну","со" "даже","она","он", ' ']

In [6]:
%%time
from time import time as t

#df = pd.read_csv('ml_tags1.csv')
#df = df[['id','client','ext','store','author','lang','country','tag','content']]

t0 = t()
wpt = nltk.TweetTokenizer()
tokenized_corpus_viz = [wpt.tokenize(re.sub('[/\nt,.qwtyuipadghjklzxcbn$&^=+"]', '', str(document))) for document in df['lemma'].str.lower()]
print('tokenize end')
tmp = []
z = 0
for j in tokenized_corpus_viz:
    d = [doc not in stop_word for doc in j]
    d = list(np.array(j)[d])
    k = 0
    dtemp = []
    for i in range(len(d)):
        #print(d[i])
        if d[i] == 'не':
            try:
                dtemp.append(d[i] + '_' + d[i+1])
                k =1
            except:
                0
        else:
            if k == 0:
                dtemp.append(d[i])
            else:
                k = 0
    d = dtemp
    tmp.append(d)
    if z%100000 ==0:
        print(z)
    z+=1
tokenized_corpus_viz = tmp
t1 = t()
print("%s: %.2g sec" % ('tokenize end: ', int(t1 - t0)))

tokenize end
0
tokenize end: : 0 sec
CPU times: user 133 ms, sys: 8.24 ms, total: 141 ms
Wall time: 139 ms


In [7]:
%%time
fastText = FastText.load('fast.model')
new_tokenize =[]
k = 0
t0 = t()
for rew in tokenized_corpus_viz:
    new_rew = []
    for word in rew:
        try:
            
            if fastText.wv.vocab[word].index > 1400:
                if fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][1] >= 0.52:
                    new_rew.append(fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][0])
            else:
                new_rew.append(word)
        except:
            try:
                if fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][1] >= 0.52:
                    new_rew.append(fastText.wv.most_similar(word, restrict_vocab=1400, topn=1)[0][0])
            except:
                0
    new_tokenize.append(new_rew)
    if k%100000 == 0:
        print(k)
        print(t() - t0)
        t0 = t()
    k+=1

0
0.403095006942749
CPU times: user 3.42 s, sys: 2.82 s, total: 6.24 s
Wall time: 3.59 s


In [8]:
%%time
bigram_transformer = Phraser.load('bi_trans.phr')
trigram_trasformer = Phraser.load('tri_trans.phr')

def tex_gen_trigram(tokenized_corpus):
    trig = []
    for text in tokenized_corpus:
        trig.append(trigram_trasformer[bigram_transformer[[word for word in text]]])
    return trig

trigram_corp = tex_gen_trigram(new_tokenize)

CPU times: user 1.54 s, sys: 141 ms, total: 1.68 s
Wall time: 1.68 s


In [9]:
%%time
w2v = Word2Vec.load('w2v.model')
new_tokenize =[]
k = 0
t0 = t()
for rew in trigram_corp:
    new_rew = []
    for word in rew:
        try:
            if w2v.wv.vocab[word].index > 1700:
                new_rew.append(model.wv.most_similar(w2v, restrict_vocab=1700, topn=1)[0][0])
            else:
                new_rew.append(word)
        except:
            0
    new_tokenize.append(new_rew)
    if k%100000 == 0:
        print(k)
        print(t() - t0)
        t0 = t()
    k+=1

0
0.00010704994201660156
CPU times: user 195 ms, sys: 16.3 ms, total: 211 ms
Wall time: 210 ms


/home/but/.local/lib/python3.5/site-packages/gensim/utils.py:491: DeprecationWarning:

Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).



In [12]:
%%time


keys = pd.read_json('keys.json')

dict_tag = joblib.load('tag.model')

nnkk = keys.cl_id.drop_duplicates()
df_attrharr = pd.DataFrame([0 for i in range(len(nnkk))], nnkk)

kknn = keys.clust1.append(keys.clust2).drop_duplicates()
df_tag = pd.DataFrame([0 for i in range(len(kknn))], kknn)

for j in range(len(trigram_corp)):
    d = {}
    d1 = {}
    
    for i in trigram_corp[j]:
        
        try:
            try:
                d[dict_tag[i][0]] += 1

            except:
                d[dict_tag[i][0]] = 1


            try:
                d[dict_tag[i][1]] += 1

            except:
                d[dict_tag[i][1]] = 1


            try:
                d1[dict_tag[i][2]] += 1
            except:
                d1[dict_tag[i][2]] = 1
        except:
            0
    
    try:
        d.pop('delete')
    except:
        0

    try:
        d1.pop('delete')
    except:
        0

    for g in d.keys():
        df_tag[j].loc[g] = d[g]
    for g in d1.keys():
        df_attrharr[j].loc[g] = d1[g]

    df_tag[j+1] = 0
    df_attrharr[j+1] = 0 
    

CPU times: user 3.59 s, sys: 47 ms, total: 3.63 s
Wall time: 3.65 s


# Основные теги по всем отзывам

In [18]:
import plotly.plotly as py
import plotly.offline as py1
import plotly.graph_objs as go

pipe = df_tag.sum(axis=1)/df_tag.sum(axis=1).sum()

labels = pipe[pipe>0.02].index
values = pipe[pipe>0.02].values

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~texpomru13/0 or inside your plot.ly account where it is named 'basic_pie_chart'


# Какие теги встречаются вместе с тегом "Дизайн"

In [33]:
disign = df_tag[list(df_tag.columns[df_tag.loc['дизайн'] > 0])]

pipe = disign.sum(axis=1)/disign.sum(axis=1).sum()

labels = pipe[pipe>0.02].index
values = pipe[pipe>0.02].values

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~texpomru13/0 or inside your plot.ly account where it is named 'basic_pie_chart'


In [ ]:
pipe = df_tag.sum(axis=1)/df_tag.sum(axis=1).sum()

labels = pipe[pipe>0.02].index
values = pipe[pipe>0.02].values

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')

# Негативные отзывы вместе с тегом Дизайн или Интерфейс

In [21]:
for i in df_tag:
    if df_tag[i].loc['негатив'] > 0 and (df_tag[i].loc['дизайн'] > 0 or df_tag[i].loc['интерфейс'] > 0):
        print(df.content.loc[i])
        print(df_tag[i][df_tag[i] != 0])

Стоит сделать окно подтверждения для аннуляции заказа! У меня возникла ситуация, когда кнопка «Аннулировать заказ» была нажата случайно и он сразу же аннулировался без всякого подтверждения типа «Вы действительно хотите аннулировать заказ» и тд! Это очень ужасно! Я потеряла заказ с 40% скидкой из-за такой нелепости и очень расстроена!
реклама      1
обращение    2
проблема     2
негатив      2
интерфейс    2
Name: 7, dtype: int64
Приложение хорошее, но все портит одна деталь. Когда смотришь статус заказа, можно случайно его аннулировать. Т.е. когда случайно нажимаешь на кнопку «Аннулировать заказ», приложение не переспрашивает «Вы точно хотите аннулировать заказ?». В результате заказ аннулирован, отменить это, как я узнал в службе доставки, нельзя. Теперь придётся ждать ещё один день. Неприятно, хоть виноват и я сам. Попрошу сделать такую возможность разработчикам.
вопрос         1
позитив        2
обращение      4
проблема       1
негатив        1
предложение    1
интерфейс      1
Nam

# Все размеченные комментарии

In [19]:

for top in pipe[pipe>0.02].index:
    for i in df_tag:
        if df_tag[i].loc[top] > 0.3:
            print(df.content.loc[i])
            print(df_tag[i][df_tag[i] != 0])
            print('---------------------------------------------------------------------')
    print(top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!'+top+'!!!!!!!!!!!!!')

Пишет нет подключения к интернет и соответственно не работает. Естественно интернет при этом есть, и все другие программы в норме
баг            2
позитив        1
обновление     1
интернет       3
не работает    2
Name: 0, dtype: int64
---------------------------------------------------------------------
В целом неплохо, но нет подписи под книгами и товарами, которые уже были приобретены. На сайте лабиринта такая информация отображается, что весьма удобно.
баг         1
позитив     1
деньги      1
подписка    1
удобно      1
Name: 11, dtype: int64
---------------------------------------------------------------------
Сделайте подтверждение отмены заказа. Пж бесит, случайный клик сделал и неделя ожидания впустую.
баг            1
деньги         1
негатив        1
предложение    1
Name: 19, dtype: int64
---------------------------------------------------------------------
Лабиринтом стала пользоваться совсем недавно. Это восторг. Книги можно забрать у себя в пригороде. В процессе получен

Name: 492, dtype: int64
---------------------------------------------------------------------
Заказываю обычно детские книги. Обидно бывает, что когда выбор подарка, то 
выбор на подарочные книги, скажем так, не фонтан. И сортировки нет хотя бы 
по авторам или жанрам. Тк шлака перерыть кучу не всегда время есть. Очень 
порадовала техподдержка
баг            1
позитив        1
обращение      1
проблема       1
негатив        1
предложение    2
не работает    1
Name: 508, dtype: int64
---------------------------------------------------------------------
Я благодарна и приятно удивлена.всегда заказывала книги в читай-городе, но 
здесь при появлении скидок ни с каким другим сайтом не сравнить.и бесплатно 
забирать при самовывозе это моё всё))
баг          1
позитив      3
обращение    1
деньги       1
негатив      1
Name: 511, dtype: int64
---------------------------------------------------------------------
Вылетает при оформлении заказа
баг         1
вылетает    1
Name: 516, dtype: int64

Name: 114, dtype: int64
---------------------------------------------------------------------
Чёрная иконка -это очень стильно решение .Спасибо большое ,теперь приложение смотрится очень свежо на рабочем столе айфона
дизайн       3
позитив      2
обращение    1
интерфейс    3
Name: 115, dtype: int64
---------------------------------------------------------------------
Насчёт приложения -5/5 
Удобно , понятно , и при этом можно купить книги со скидкой сразу . 
Огромное количество акций , нравиться рулетка( можно выиграть или доставку бесплатную или скидку на следующий заказ и тд) . 
Отслеживание заказа есть ( также информируют на почту и дублируют на телефон, по приезду на точку выдачи). 
Существенные плюсы описал. 
Работает стабильно. 
Поддержка есть практически в любом городе (где есть пункт выдачи) , 24/7 . 

Заказывал подруге бокс с 7 книгами, на моем акк-е , как контактное лицо указал ее ( по приходу ей пришло смс о том что можно забрать) . Заказы проверяются перед отдачей клиенту 

---------------------------------------------------------------------
"мрачный дизайн" как-то не очень, а в остальном всё ок! Спасибо!
дизайн       1
позитив      2
обращение    1
негатив      1
Name: 672, dtype: int64
---------------------------------------------------------------------
Новая иконка и открытие просто топ. Но дизайн нужно привести к одному 
стилю, потому что цвета идут как-то в перемешку. Черный, красный, 
темно-синий, оранжевый... Оформление заказа стало намного удобнее. Есть баг 
в выполненных заказах. Убираешь галку с фильтра аннулирование, он 
применяется. Снова заходишь - фильтр убирается
баг          2
дизайн       6
позитив      2
интерфейс    1
удобно       1
Name: 673, dtype: int64
---------------------------------------------------------------------
Отличное приложение, работает стабильнее, чем сайт. Дизайн тоже понравился. 
Есть нюанс, ещё до обновления стал глючить каталок с подарками, добавляю 
книги, обновляю корзину, чтобы увидеть итого, количество, наим

Name: 122, dtype: int64
---------------------------------------------------------------------
Очень нравится мне этот магазин
позитив    1
Name: 123, dtype: int64
---------------------------------------------------------------------
Очень удобно!
позитив    1
удобно     1
Name: 124, dtype: int64
---------------------------------------------------------------------
Хайли рекомменд!отличнейшее приложение для покупки любимых и новых книг!все удобно,подробно и понятно!
позитив    2
деньги     1
удобно     1
Name: 125, dtype: int64
---------------------------------------------------------------------
Отличное приложение! Все удобно и просто!как с навигацией, так и оплатой! Спасибо!
позитив      2
обращение    1
деньги       1
удобно       1
Name: 126, dtype: int64
---------------------------------------------------------------------
Отличное приложение, жаль что склады пустуют, очень много позиций нет в наличии, к сожалению.
баг        1
позитив    1
функция    1
Name: 127, dtype: int64
---

...

---------------------------------------------------------------------
УЖАСНО!!! Книги пришли все битые, порезанные, в хрен знает, каком 
состоянии!!!! Никогда больше заказывать тут не буду!!! Купила книгу за 700 
рублей, а она пришла с равной обложкой, мятыми листами и сломонны 
переплетом. Как так можно?!?!?!?
вопрос        2
деньги        1
негатив       3
интерфейс     1
отправлять    2
Name: 662, dtype: int64
---------------------------------------------------------------------
Отличное приложение, но часто бывает, что нет в наличии нужных книг. Они 
или вообще не появляются или появляются,но спустя очень долгое время.
позитив     1
проблема    1
негатив     1
тормозит    1
Name: 665, dtype: int64
---------------------------------------------------------------------
Я не первая, но всё-таки повторюсь, новый дизайн, цвет отталкивает. Верните 
обратно красный цвет, который несёт тепло. А не черный-мрачность, 
безысходность, чёрствость, траур. Или вы решили закрыться?
вопрос        1


сайте, а не в приложении. Сайт обычный, иногда даже удобнее.
дизайн         3
позитив        2
предложение    1
удобно         1
Name: 532, dtype: int64
---------------------------------------------------------------------
всё отлично рекомендую!всё очень доступно,быстро и удобно!
позитив        1
предложение    1
удобно         1
Name: 537, dtype: int64
---------------------------------------------------------------------
Почему не выбирается пункт самовывоза? Когда выбираю, приложение 
вылетает!!!!
баг            1
вопрос         2
проблема       1
предложение    1
вылетает       1
Name: 542, dtype: int64
---------------------------------------------------------------------
Лично мне неудобно листать отложенные книги. У меня их очень много, книги я 
обожаю. Бывает частенько, что пока листаешь, приложение слетает и 
приходится начинать сначала. Мне это неудобно. В остальном все хорошо. Ну и 
именно к системе продаж претензия такая, что перед скидками завышают 
изначальную цену, я уже 

Name: 129, dtype: int64
---------------------------------------------------------------------
Удобное, не глючит и красивое )
дизайн     1
позитив    2
удобно     1
Name: 130, dtype: int64
---------------------------------------------------------------------
Приложение отличное! Очень удобно! Огромное спасибо)
позитив      2
обращение    1
удобно       1
Name: 133, dtype: int64
---------------------------------------------------------------------
Все удобно и легко
удобно    1
Name: 134, dtype: int64
---------------------------------------------------------------------
Приложение отличное! Теперь заказы делать гораздо удобней!
позитив    1
удобно     1
Name: 138, dtype: int64
---------------------------------------------------------------------
Магазин с широким ассортиментом и удобной службой доставки. Рекомендую.
позитив        1
предложение    1
удобно         1
Name: 139, dtype: int64
---------------------------------------------------------------------
Отличное, очень удобное прил

Быстро и удобно.
удобно    1
Name: 374, dtype: int64
---------------------------------------------------------------------
Очень удобное приложение, спасибо. И про черный цвет правильно пишут, 
красный больше привлекал внимание.
баг          1
дизайн       2
позитив      2
обращение    1
удобно       1
Name: 376, dtype: int64
---------------------------------------------------------------------
В приложении нет подкаталогов, например, в детском творчестве. Показывается 
сразу все 5000 наименований. Нет фильтров. Через браузер удобнее искать, 
получается.
баг       1
удобно    1
Name: 377, dtype: int64
---------------------------------------------------------------------
Мне очень нравится, магазин давно любимый, теперь с приложением стало 
намного удобнее. Но честно говоря коп ный цвет значка нравился больше.
дизайн     2
позитив    4
удобно     1
Name: 380, dtype: int64
---------------------------------------------------------------------
Приложение супер, удобное
позитив    2
удобно 

дизайн     1
позитив    3
удобно     1
Name: 754, dtype: int64
---------------------------------------------------------------------
Классный магазин, очень удобный правда сбои что-то в приложении последнее 
время на последнем Андроиде и ещё очень прошу писать или уведомлять 
вследующий раз, что посылку третьим лицам по покупке безналичным расчетом 
не выдаётся просто это нигде не было написано, а так вы супер)
баг          2
позитив      3
обращение    2
деньги       1
проблема     2
удобно       1
счет         1
Name: 756, dtype: int64
---------------------------------------------------------------------
Меня все устраивает.Очень удобго заказывать,но хотелось бы побольше 
технических книг.
позитив        1
предложение    1
удобно         1
Name: 757, dtype: int64
---------------------------------------------------------------------
Отличное приложение, удобно оформлять заказ 😻
позитив    2
удобно     1
Name: 758, dtype: int64
----------------------------------------------------------